# Construire un EAF à partir d'un fichier de sous-titres issu de Whisper
- le traitement est basé sur un SRT
- chaque LINGUISTIC_TYPE_REF des TIER doit correspondre à un LINGUISTIC_TYPE_ID dans les LINGUISTIC_TYPE


In [13]:
import lxml.etree as ET
import codecs
import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [14]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

## Sélection du fichier SRT

In [21]:
jsonReps=glob.glob("/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/*/*/"%nMachine)
jsonReps

['/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 13/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 14/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 16/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 17/',
 '/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACH

## Construction de la coquille
- racine ANNOTATION_DOCUMENT
    - HEADER minimal avec MEDIA_DESCRIPTOR et PROPERTY
    - TIME_ORDER
    - TIER 
    - LINGUISTIC_TYPE

In [39]:
def makeDocument():
    ANNOTATION_DOCUMENT=ET.Element("ANNOTATION_DOCUMENT", AUTHOR="", DATE="", FORMAT="3.0", VERSION="3.0")
    HEADER=ET.SubElement(ANNOTATION_DOCUMENT, "HEADER", MEDIA_FILE="", TIME_UNITS="milliseconds")
    # la balise MEDIA_DESCRIPTOR fait bugger ELAN
    PROPERTY=ET.SubElement(HEADER, "PROPERTY", NAME="lastUsedAnnotationId")
    TIME_ORDER=ET.SubElement(ANNOTATION_DOCUMENT, "TIME_ORDER")
    act1TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper",
                           LINGUISTIC_TYPE_REF="Whisper",PARTICIPANT="Acteur1", TIER_ID="Acteur1")
    act2TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur2", TIER_ID="Acteur2")
    act3TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur3", TIER_ID="Acteur3")
    act4TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur4", TIER_ID="Acteur4")
    LINGUISTIC_TYPE=ET.SubElement(ANNOTATION_DOCUMENT, "LINGUISTIC_TYPE", GRAPHIC_REFERENCES="false",
                                  LINGUISTIC_TYPE_ID="Whisper", TIME_ALIGNABLE="true")
    return ANNOTATION_DOCUMENT

### Header
- header
     - media_descriptor
     - property
 

### Time order for Time slots
- time order
    - time slot


### Tiers for Annotations
- tier
    - annotation
        - alignable annotation
            - annotation value


### Linguistic type

<div style="page-break-after: always;"></div>

## Conversion des sous-titres vers EAF

### Récupération des données SRT

In [23]:
def getSeconds(time):
    h, m, s = [float(i) for i in time.replace(",",".").split(':')]
    result=((60*h+m)*60+s)*1000
    return str(int(result))

def getSubtitles(lignes):
    result=[]
    currentState="start"
    currentNum=1
    for l in lignes:
        if currentState=="start" and str(currentNum)==l:
            num=l
            currentState="time"
        elif currentState=="time":
            t1, t2=l.split(" --> ")
            t1=getSeconds(t1)
            t2=getSeconds(t2)
            currentState="text"
        elif currentState=="text":
            text=l
            currentState="skip"
        elif currentState=="skip" and l=="":
            currentState="start"
            currentNum+=1
            result.append({"num":num, "start":t1, "end":t2, "text":text})
    return result        

### Insertion des sous-titres dans la coquille

In [24]:
def fillActor(actTIER,subtitles):
    for s in subtitles:
        ANNOTATION=ET.SubElement(actTIER, "ANNOTATION")
        aid="au"+s["num"]
        ALIGNABLE_ANNOTATION=ET.SubElement(ANNOTATION, "ALIGNABLE_ANNOTATION", 
                                           ANNOTATION_ID=aid, 
                                           TIME_SLOT_REF1=aid+s["start"], 
                                           TIME_SLOT_REF2=aid+s["end"]
                                          )
        ANNOTATION_VALUE=ET.SubElement(ALIGNABLE_ANNOTATION, "ANNOTATION_VALUE")
        ANNOTATION_VALUE.text=s["text"]
        TIME_SLOT=ET.SubElement(TIME_ORDER, "TIME_SLOT", 
                                TIME_SLOT_ID=aid+s["start"], 
                                TIME_VALUE=s["start"], 
                               )
        TIME_SLOT=ET.SubElement(TIME_ORDER, "TIME_SLOT", 
                                TIME_SLOT_ID=aid+s["end"], 
                                TIME_VALUE=s["end"], 
                               )

## Exportation du fichier EAF

In [40]:
def makeEAF(fSRT):
    with codecs.open(fSRT, "r", encoding="utf8") as inFile:
        lignesSRT=[l.strip() for l in inFile.readlines()]
    if lignesSRT[0].endswith("1"): lignesSRT[0]="1"
    else: print("vérifier le fichier SRT")
    subtitles=getSubtitles(lignesSRT)
    for actTIER in actTIERS:
        fillActor(actTIER,subtitles)
    EAF=ET.ElementTree(ANNOTATION_DOCUMENT)
    EAF.write(fEAF, pretty_print=True, encoding='utf-8', xml_declaration=True)
    return

In [47]:
for repJson in jsonReps:
    print(repJson)
    ANNOTATION_DOCUMENT=ET.Element("ANNOTATION_DOCUMENT", AUTHOR="", DATE="", FORMAT="3.0", VERSION="3.0")
    HEADER=ET.SubElement(ANNOTATION_DOCUMENT, "HEADER", MEDIA_FILE="", TIME_UNITS="milliseconds")
    # la balise MEDIA_DESCRIPTOR fait bugger ELAN
    PROPERTY=ET.SubElement(HEADER, "PROPERTY", NAME="lastUsedAnnotationId")
    TIME_ORDER=ET.SubElement(ANNOTATION_DOCUMENT, "TIME_ORDER")
    act1TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper",
                           LINGUISTIC_TYPE_REF="Whisper",PARTICIPANT="Acteur1", TIER_ID="Acteur1")
    act2TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur2", TIER_ID="Acteur2")
    act3TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur3", TIER_ID="Acteur3")
    act4TIER=ET.SubElement(ANNOTATION_DOCUMENT, "TIER", ANNOTATOR="Whisper", LINGUISTIC_TYPE_REF="Whisper",
                           PARTICIPANT="Acteur4", TIER_ID="Acteur4")
    LINGUISTIC_TYPE=ET.SubElement(ANNOTATION_DOCUMENT, "LINGUISTIC_TYPE", GRAPHIC_REFERENCES="false",
                                  LINGUISTIC_TYPE_ID="Whisper", TIME_ALIGNABLE="true")
    try:
        fSRT=glob.glob(repJson+"*-tours.srt")[0]
    except:
        print("no SRT",repJson)
        continue
    fEAF=repJson+repJson.split("/")[-2]+".eaf"
    makeEAF(fSRT)
    print(fEAF)
    print()


/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/
no SRT /Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_10/
/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/
no SRT /Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/BerloinSalome/enregistrement_9/
/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/
/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10/ECOLE PRIMAIRE LEOPOLD LACHAIZE 10.eaf

/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 13/
no SRT /Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/iMac/Json/ClavereauDarrieumerlouMarcheix/ECOLE PRIMAIRE LEOPOLD LACHAIZE 13/
/Users/gilles/sDrive/Cours/Bor

In [460]:
ding()